# Storage formats

## Download CSV data (subset) from S3

TODO

## Download Parquet data (subset) from S3

TODO

## Understanding Parquet

TODO

## Comparing performance

TODO

## Convert from CSV to Parquet

TODO

## Exercises

TODO